In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [ ]:
class Network(nn.Module):#Extends all features of Module(bulit in class)to oyr network class
    def __init__(self):#Defines the type of layers as part of the object
        super(Network,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=(1,1))
        self.conv2=nn.Conv2d(in_channels=6,out_channels=10,kernel_size=5,stride=(1,1))
        
        self.fc1=nn.Linear(in_features=10*4*4,out_features=90)
        self.out=nn.Linear(in_features=90,out_features=10)
    
    
    def forward(self,t):
        
        #Conv layer 1
        t=self.conv1(t)#Implements the forward pass through predefined functions in the nn.Module class
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        
        #Conv layer 2
        t=self.conv2(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        
        #Fc layer 1
        t=t.reshape(-1,10*4*4)
        t=self.fc1(t)
        t=F.relu(t)
        
        #Final softmax layer
        t=self.out(t)
        #t=F.softmax(t,dim=1),No need to do this cross entropy does it by itself
        
        return t

In [ ]:
net=torch.load('ConvNet minibatch MNIST.pth')
net.eval()

In [ ]:
img1=plt.imread('Downloads/img/6i.jpeg')

In [ ]:
plt.imshow(img1)

In [ ]:
np.array(img1)
imgt=torch.from_numpy(img1)

In [ ]:
type(img1)

In [ ]:
#pred=net(255-imgt.unsqueeze(0).unsqueeze(0).float())

In [ ]:
print(pred.argmax(dim=1))

In [ ]:
def preprocess(image):
    img2=img1[:,:,0]/3+img1[:,:,1]/3+img1[:,:,2]/3
    v_s = 0
    v_e = img2.shape[0]

    h_s = 0
    h_e = img2.shape[1]

    for i in range(img2.shape[0]):
        if np.min(img2[i, :]) > 30:
            v_s += 1
        else:
            break
    for i in reversed(range(img2.shape[0])):
        if np.min(img2[i, :]) > 30:
            v_e -= 1
        else:
            break
    for i in range(img2.shape[1]):
        if np.min(img2[:, i]) > 30:
            h_s += 1
        else:
            break
    for i in reversed(range(img2.shape[1])):
        if np.min(img2[:, i]) > 30:
            h_e -= 1
        else:
            break

    # Slicing that image
    img3 = img2[v_s:v_e, h_s:h_e]
     # Maintaining the aspect ratio
    ih = h_e - h_s
    iv = v_e - v_s
    if ih > iv:
        p = int((ih - iv) / 2)
        img4 = np.pad(np.array(img3), ((p, p), (0, 0)), 'constant', constant_values=(255, 255))
    else:
        p = int((iv - ih) / 2)
        img4 = np.pad(np.array(img3), ((0, 0), (p, p)), 'constant', constant_values=(255, 255))

    img5 = cv2.resize(img4, (20, 20))
    return img5

In [ ]:
i=preprocess(img1)

In [ ]:
plt.imshow(i,cmap='gray')

In [ ]:
def find_centroid(img):
    x_cm=0
    y_cm=0
    img=255-img
    for i in range(1,img.shape[0]+1):
        y_cm+=np.sum(i*img[:,i-1])
    for i in range(1,img.shape[1]+1):
        x_cm+=np.sum(i*img[i-1,:])
    y_cm=int(y_cm/np.sum(img))
    x_cm=int(x_cm/np.sum(img))
    return x_cm,y_cm

In [ ]:
x,y= find_centroid(i)
print(x)
print(y)

In [ ]:
def padder(img,x,y):

    k=10-y
    h=10-x
        
        
        
    img = np.pad(np.array(img), ((4+h, 4-h), (4+k, 4-k)), 'constant', constant_values=(255, 255))
    return(img)    

In [ ]:
i1=padder(i,x,y)
plt.imshow(i1)

In [ ]:
x1,y1=find_centroid(i1)
print(x1,y1)

In [ ]:
i_un = np.pad(i, ((4, 4), (4, 4)), 'constant', constant_values=(255, 255))
input=torch.from_numpy(255-i_un)
net=torch.load('ConvNet MNIST.pth')
net.eval()

pred=net(input.unsqueeze(0).unsqueeze(0).float())
#print(pred)

print("Prediction:"+str(int(pred.argmax(dim=1))))


In [ ]:
input=torch.from_numpy(255-i1)
net=torch.load('ConvNet MNIST.pth')
net.eval()

pred=net(input.unsqueeze(0).unsqueeze(0).float())
#print(pred)

print("Prediction:"+str(int(pred.argmax(dim=1))))
